In [1]:
import numpy as np
import pandas as pd
import re
import pickle
from pprint import pprint
import datetime
import math
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json
from bson import json_util

from mongoengine import *

In [2]:
from flask import Flask,jsonify, request, send_from_directory, url_for
app = Flask(__name__)
import pickle
import numpy as np
from itertools import chain

In [428]:
disconnect('mongo://mongo:27017')

In [3]:
conn = connect("local",host="mongodb://localhost:27017",alias='default')

if(0):
    disconnect('mongo://mongo:27017')

In [4]:
if(0):
    conn_aws = connect("local",host="mongodb://ec2-13-233-196-75.ap-south-1.compute.amazonaws.com:27017",alias='default')

if(0):
    disconnect('mongo://mongo:27017')

In [90]:
str_menu=pickle.load(open('full_menu_main.pkl','rb'))
if(0):
    pickle.dump(str_menu,open('half_menu_main.pkl','wb'))
restautant_id = 'BNGHSR0001'

final_list_json=[]

str_menu['price']=str_menu['price'].apply(lambda x: x.split('/')[0])

In [7]:
def chainer(string):
    return list(chain.from_iterable(string.str.split('/')))

def str_2(number):
    string = str(number)
    if(len(string)==1):
        return '0'+string
    else:
        return string

In [91]:
final_list_json=[]
for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['food_list']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['food_list'].append({})
            final_list_json[n]['sub_category'][p]['food_list'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['food_list'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['food_list'][q]['price']=food_dict[1]['price']
        

In [92]:
pickle.dump(final_list_json,open('final_usable_json.pkl','wb'))

final_list_json=pickle.load(open('final_usable_json.pkl','rb'))

# All Mongo Classes

In [130]:

class TableOrder(Document):
    pass


class User(Document):
    dine_in_history = DynamicField()
    current_table_id = StringField()
    personal_cart = ListField(ReferenceField(TableOrder))
    meta = {'allow_inheritance': True}


class TempUser(User):
    pass


class RegisteredUser(User):
    name = StringField(required=True)
    email_id = StringField(required=True)
    phone_no = StringField()
    tempuser_ob = ReferenceField(TempUser)


class TempUser(User):
    unique_id = StringField(required=True)
    reguser_ob = ReferenceField(RegisteredUser)


class Assistance(Document):
    pass


class Staff(Document):
    name = StringField()
    assistance_history = MapField(ListField(ReferenceField(Assistance)))
    order_history = MapField(ListField(ReferenceField(TableOrder)))

    def to_my_mongo(self):
        data = self.to_mongo()

        return data


class Assistance(Document):
    types = ['water', 'help', 'cutlery', 'tissue', 'cleaning', 'menu', 'ketchup']
    table = StringField()
    user = ReferenceField(User)
    assistance_type = StringField(choices=types)
    timestamp = DateTimeField()
    accepted_by = ReferenceField(Staff, default=None)
    meta = {'strict': False}

    def to_my_mongo(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        data['table'] = Table.objects(assistance_reqs__in=[self.id])[0].name

        return data

    def to_json(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        data['table'] = Table.objects(assistance_reqs__in=[self.id])[0].name
        return json_util.dumps(data)


class FoodOptionsMod(EmbeddedDocument):
    options = DictField()
    choices = ListField()


class FoodItemMod(EmbeddedDocument):
    food_id = StringField()
    name = StringField()
    description = StringField()
    price = StringField()
    instructions = StringField()
    quantity = IntField()
    status = StringField(choices=['queued', 'cooking', 'completed'])
    food_options = EmbeddedDocumentField(FoodOptionsMod)


class Order(Document):
    placed_by = ReferenceField(User)
    food_list = ListField(EmbeddedDocumentField(FoodItemMod))
    status = StringField(choices=['queued', 'juststarted', 'cooking', 'almostdone', 'completed'], default='queued')

    def fetch_food_item(self, food_id):
        for food_item in self.food_list:
            if food_item.food_id == food_id:
                return food_item.to_json()
        return "Food item not found"


class TableOrder(Document):
    table = StringField()
    table_id = StringField()
    orders = ListField(ReferenceField(Order))
    status = StringField(choices=['queued', 'juststarted', 'cooking', 'almostdone', 'completed'], default='queued')
    timestamp = DateTimeField(default=datetime.datetime.now())

    def to_my_mongo(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        for key, order in enumerate(self.orders):
            data['orders'][key] = json_util.loads(self.orders[key].to_json())

        return data

    def to_json(self):
        data = self.to_mongo()
        data['timestamp'] = str(data['timestamp'])
        for key, order in enumerate(self.orders):
            data['orders'][key] = json_util.loads(self.orders[key].to_json())
        return json_util.dumps(data)


class Table(Document):
    name = StringField(required=True)
    seats = IntField(required=True)
    staff = ListField(ReferenceField(Staff))
    users = ListField(ReferenceField(User))
    no_of_users = IntField()
    table_orders = ListField(ReferenceField(TableOrder))
    assistance_reqs = ListField(ReferenceField(Assistance))
    meta = {'strict': False}

    def to_my_mongo(self):
        data = self.to_mongo()
        for key, table_order in enumerate(self.table_orders):
            data['table_orders'][key] = self.table_orders[key].to_my_mongo()
        for key, ass_req in enumerate(self.assistance_reqs):
            data['assistance_reqs'][key] = self.assistance_reqs[key].to_my_mongo()

        return data


class FoodOptions(EmbeddedDocument):
    options = DictField()
    choices = ListField()


class FoodItem(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    price = StringField(required=True)
    food_options = EmbeddedDocumentField(FoodOptions)

    def to_my_mongo(self):
        data = self.to_mongo()
        if self.food_options:
            data['food_options'] = self.food_options.to_mongo()

        return data

    def to_json(self):
        data = self.to_mongo()
        if self.food_options:
            data['food_options'] = self.food_options.to_mongo()

        return json_util.dumps(data)


class SubCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    food_list = ListField(ReferenceField(FoodItem), required=True)

    def to_my_mongo(self):
        data = self.to_mongo()
        for key, food_item in enumerate(self.food_list):
            data['food_list'][key] = self.food_list[key].to_my_mongo()

        return data


class MainCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    sub_category = ListField(ReferenceField(SubCategory), required=True)

    def to_my_mongo(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.sub_category):
            data['sub_category'][key] = self.sub_category[key].to_my_mongo()

        return data


def check_exists(order_id, order_list):
    for n, order in enumerate(order_list):
        if order_id == order['_id']:
            return n
    return -1


class Restaurant(Document):
    name = StringField(required=True)
    restaurant_id = StringField(required=True)
    menu = ListField(ReferenceField(MainCategory), required=True)
    address = StringField()
    tables = ListField(ReferenceField(Table))
    staff = ListField(ReferenceField(Staff))
    table_orders = ListField(ReferenceField(TableOrder))
    assistance_reqs = ListField(ReferenceField(Assistance))

    def to_json(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.menu):
            data['menu'][key] = self.menu[key].to_my_mongo()
        for key, staff in enumerate(self.staff):
            data['staff'][key] = self.staff[key].to_my_mongo()
        for key, table in enumerate(self.tables):
            data['tables'][key] = self.tables[key].to_my_mongo()

        return json_util.dumps(data)

    def fetch_order_lists(self):
        q_list = []
        cook_list = []
        comp_list = []
        for table_order_ob in self.table_orders:
            tabord_dict = json_util.loads(table_order_ob.to_json())
            if tabord_dict['status'] == 'completed':
                comp_list.append(tabord_dict)
                break
            for order in tabord_dict['orders']:
                for food_item in order['food_list']:
                    if food_item['status'] == 'queued':
                        update_list = q_list
                        index = check_exists(tabord_dict['_id'], update_list)
                        if index >= 0:
                            order_index = check_exists(order['_id'], update_list[index]['orders'])
                            if order_index >= 0:
                                update_list[index]['orders'][order_index]['food_list'].append(food_item)
                            else:
                                o_app_dict = {k: order[k] for k in ['_id', 'placed_by', 'status']}
                                o_app_dict['food_list'] = []
                                o_app_dict['food_list'].append(food_item)
                                update_list[index]['orders'].append(o_app_dict)

                        else:
                            t_app_dict = {k: tabord_dict[k] for k in ['_id', 'table', 'status', 'timestamp']}
                            o_app_dict = {k: order[k] for k in ['_id', 'placed_by', 'status']}
                            o_app_dict['food_list'] = []
                            o_app_dict['food_list'].append(food_item)
                            t_app_dict['orders'] = []
                            t_app_dict['orders'].append(o_app_dict)
                            update_list.append(t_app_dict)
                    elif food_item['status'] == 'cooking':
                        update_list = cook_list
                        index = check_exists(tabord_dict['_id'], update_list)
                        if index >= 0:
                            order_index = check_exists(order['_id'], update_list[index]['orders'])
                            if order_index >= 0:
                                update_list[index]['orders'][order_index]['food_list'].append(food_item)
                            else:
                                o_app_dict = {k: order[k] for k in ['_id', 'placed_by', 'status']}
                                o_app_dict['food_list'] = []
                                o_app_dict['food_list'].append(food_item)
                                update_list[index]['orders'].append(o_app_dict)

                        else:
                            t_app_dict = {k: tabord_dict[k] for k in ['_id', 'table', 'status', 'timestamp']}
                            o_app_dict = {k: order[k] for k in ['_id', 'placed_by', 'status']}
                            o_app_dict['food_list'] = []
                            o_app_dict['food_list'].append(food_item)
                            t_app_dict['orders'] = []
                            t_app_dict['orders'].append(o_app_dict)
                            update_list.append(t_app_dict)
                    elif food_item['status'] == 'completed':
                        update_list = comp_list
                        index = check_exists(tabord_dict['_id'], update_list)
                        if index >= 0:
                            order_index = check_exists(order['_id'], update_list[index]['orders'])
                            if order_index >= 0:
                                update_list[index]['orders'][order_index]['food_list'].append(food_item)
                            else:
                                o_app_dict = {k: order[k] for k in ['_id', 'placed_by', 'status']}
                                o_app_dict['food_list'] = []
                                o_app_dict['food_list'].append(food_item)
                                update_list[index]['orders'].append(o_app_dict)

                        else:
                            t_app_dict = {k: tabord_dict[k] for k in ['_id', 'table', 'status', 'timestamp']}
                            o_app_dict = {k: order[k] for k in ['_id', 'placed_by', 'status']}
                            o_app_dict['food_list'] = []
                            o_app_dict['food_list'].append(food_item)
                            t_app_dict['orders'] = []
                            t_app_dict['orders'].append(o_app_dict)
                            update_list.append(t_app_dict)
        return json_util.dumps({"queue": q_list, "cooking": cook_list, "completed": comp_list})

# Generating data

In [280]:
Restaurant.objects[0].fetch_order_lists()

'{"queue": [{"_id": {"$oid": "5e8ae5409c69d718046f4883"}, "table": "table11", "status": "queued", "timestamp": "2020-04-06 13:46:00.728000", "orders": [{"_id": {"$oid": "5e8ae5409c69d718046f4885"}, "placedby": {"$oid": "5e8a228f9c69d718046f4874"}, "status": "queued", "foodlist": [{"food_id": "5e8a228f9c69d718046f480a", "name": "MUTTON GHEE ROAST", "description": "Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee", "price": "370", "instructions": "Cook with love", "quantity": 1, "status": "queued"}, {"food_id": "5e8a228e9c69d718046f47de", "name": "FRENCH FRIES", "description": "French Fries Tossed With Hoc Signature Cajun Spice Seasoning", "price": "190", "instructions": "no", "quantity": 1, "status": "queued"}]}]}, {"_id": {"$oid": "5e8ae5419c69d718046f4887"}, "table": "table15", "status": "queued", "timestamp": "2020-04-06 13:46:01.865000", "orders": [{"_id": {"$oid": "5e8ae5419c69d718046f4888"}, "placedby": {"$oid": "5e8a228f9c69d718046f4872"}, "status": "queued", "f

In [44]:
# pickle.dump(final_list_json,open('final_usable_json.pkl','wb'))

final_list_json=pickle.load(open('final_usable_json.pkl','rb'))

In [6]:
#RANDOM Functions
def random_table():
    return all_table_list[np.random.randint(len(all_table_list))]
def random_user():
    return user_list[np.random.randint(len(user_list))]
def random_food():
    return all_food_list[np.random.randint(len(all_food_list))]
def random_food_list():
    randomfl=[]
    for i in range(np.random.randint(2,9)):
        randomfl.append(random_food())
    return list(set(randomfl))

In [96]:
def user_scan(table_id, unique_id, email_id='dud'):
    scanned_table = Table.objects.get(id=table_id)
    if email_id == 'dud':
        temp_user = TempUser.objects.filter(unique_id=unique_id)
        if len(temp_user) > 0:
            temp_user[0].update(set__current_table_id=str(scanned_table.id))
            scanned_table.update(push__users=temp_user[0].to_dbref())
            return temp_user[0].id
        else:
            temp_user = TempUser(unique_id=unique_id, current_table_id=str(scanned_table.id)).save()
            scanned_table.update(push__users=temp_user.to_dbref())
            return temp_user.id
    else:
        reg_user = RegisteredUser.objects.filter(email_id=email_id)[0]
        scanned_table.update(push__users=reg_user)
        reg_user.update(set__current_table_id=str(scanned_table.id))
        scanned_table.update(inc__no_of_users=1)

In [97]:
def custom_splitter(text):
    types = []
    full_splits = re.split('[/]',text)
    for n,spl in enumerate(full_splits):
        if(n==0):
            types.append(re.search('[a-zA-Z]+',spl.strip().split()[-1]).group())
        elif(n==len(full_splits)-1):
            types.append(re.search('[a-zA-Z]+',spl.strip().split()[0]).group())
        else:
            types.append(re.search('[a-zA-Z]+',spl).group())
    return types

In [131]:
#House of commons
if(1):
    FoodItem.drop_collection()
    SubCategory.drop_collection()
    MainCategory.drop_collection()
    Restaurant.drop_collection()
    Table.drop_collection()
    TempUser.drop_collection()
    RegisteredUser.drop_collection()
    Server.drop_collection()
    Assistance.drop_collection()
    TableOrder.drop_collection()
    Order.drop_collection()
    User.drop_collection()

    main_cat_list = []
    for p,main_cat in enumerate(final_list_json):
        sub_cat_list = []
        for q,sub_cat in enumerate(main_cat['sub_category']):
            food_list = []
            for r,food in enumerate(sub_cat['food_list']):
                food_list.append(FoodItem(name=food['name'],description = food['description'],price=food['price']).save())
            sub_cat_list.append(SubCategory(name=sub_cat['name'],description=sub_cat['description'],food_list = food_list).save())
        main_cat_list.append(MainCategory(name=main_cat['name'],description = main_cat['description'],sub_category = sub_cat_list).save())

    houseofcommons=Restaurant(name = 'House of Commons',restaurant_id='BNGHSR0001',menu = main_cat_list,address='')
    houseofcommons.save()

    table_list=[]
    for n in range(1,16):
        table_list.append(Table(name='table'+str(n),seats = 6,no_of_users=0).save().to_dbref())

    Restaurant.objects[0].update(set__tables=table_list)
    akshay = TempUser(unique_id = "hfirnivnhhwocn34534no34n34r")

    akshay.save()

    RegisteredUser(name='Akshay',email_id='akshay.dn5@gmail.com',phone_no='8660961089').save()

    RegisteredUser.objects(name='Akshay',email_id='akshay.dn5@gmail.com',phone_no='8660961089').update(set__tempuser_ob=akshay.to_dbref())

    user_scan(str(table_list[2].id),'hidnfvidfkvmsdmv')

    user_scan(str(table_list[2].id),'hidnfvidvdff1kvmsadmv')

    user_scan(str(table_list[2].id),'hidnfvidvdffaskvmsadmv')

    user_scan(str(table_list[2].id),'hidnfvidvdffvf1kvmsadmv')

    user_scan(str(table_list[2].id),'',email_id='akshay.dn5@gmail.com')

In [133]:
#Craeting INDIGO XP
if(1):
    main_cat_list = []
    for p,main_cat in enumerate(final_list_json):
        sub_cat_list = []
        for q,sub_cat in enumerate(main_cat['sub_category']):
            food_list = []
            for r,food in enumerate(sub_cat['food_list']):
                food_list.append(FoodItem(name=food['name'],description = food['description'],price=food['price']).save())
            sub_cat_list.append(SubCategory(name=sub_cat['name'],description=sub_cat['description'],food_list = food_list).save())
        main_cat_list.append(MainCategory(name=main_cat['name'],description = main_cat['description'],sub_category = sub_cat_list).save())

    Restaurant(name='Indigo XP',restaurant_id="BNGHSR0002",menu = main_cat_list).save()

In [134]:
for food_ob in FoodItem.objects:
    if re.search('/', food_ob.price):
        if re.search('/', food_ob.name):
            options = custom_splitter(food_ob.name)

        elif re.search('/', food_ob.description):
            options = custom_splitter(food_ob.description)
        prices = food_ob.price.split('/')
        if len(options) == len(prices):
            options = {k: v for k, v in zip(options, prices)}
            FoodItem.objects.get(id=food_ob.id).update(set__food_options=FoodOptions(options=options))
        else:
            left_out = food_ob.id
left_out_options = {k: v for k, v in
                    zip(custom_splitter(' Vegetables/chicken/prawns, Served'), '250/280/300'.split('/'))}
FoodItem.objects.get(id=left_out).update(
    set__food_options=FoodOptions(options=left_out_options, choices=['Red', 'Green']))


1

In [135]:
user_list=[]
for obj in User.objects:
    user_list.append(str(obj.id))

all_table_list=[]
for obj in Table.objects:
    all_table_list.append(str(obj.id))
    
all_food_list=[]
for obj in FoodItem.objects:
    all_food_list.append(str(obj.id))

In [136]:
try:
    if(names):
        pass
except:
    names=pickle.load(open('indian_names.pkl','rb'))
    girls_name=names[0]
    boys_name=names[1]

In [137]:
if(1):
    total = {}
    sers=[]
    tabs=[]
    for n,table in enumerate(Table.objects):
    #     print(math.floor(n/1.5))
        sers.append(math.floor(n/1.5))
        tabs.append(n)
        if((n+1)%3==0):
            total[tuple(set(sers))]=tabs
            sers=[]
            tabs=[]


    if(1):#assiging tables to servers
        for i in range(10):
            if(i<5):
                staff=Staff(name=girls_name[np.random.randint(len(girls_name))]).save()
                Restaurant.objects[0].update(push__staff=staff.to_dbref())
            else:
                staff=Staff(name=boys_name[np.random.randint(len(boys_name))]).save()
                Restaurant.objects[0].update(push__staff=staff.to_dbref())

    for key in total.keys():
        for tab in total[key]:
            Table.objects[tab].update(push__staff=Staff.objects[key[0]].to_dbref())

        for tab in total[key]:
            Table.objects[tab].update(push__staff=Staff.objects[key[1]].to_dbref())

# Ordering updating and assistance requests

In [107]:
def food_embed(food_dict):
    json_dict = json_util.loads(FoodItem.objects(id=food_dict['food_id']).exclude('id')[0].to_json())
    json_dict['food_id'] = food_dict['food_id']
    json_dict['quantity'] = food_dict['quantity']
    json_dict['instructions'] = food_dict['instructions']
    json_dict['status'] = 'queued'
    option_id = choice_id = ''
    try:
        option = np.random.choice(list(json_dict['food_options']['options'].keys()))
        option_pair = {option: json_dict['food_options']['options'][option]}
        json_dict['food_options']['options'] = option_pair
        option_id = option.lower()
    except:
        pass
    try:
        choice = np.random.choice(list(json_dict['food_options']['choices']))
        json_dict['food_options']['choices'] = [choice]
        choice_id = choice.lower()
    except:
        pass
    if option_id != '':
        if choice_id != '':
            json_dict['food_id'] = food_dict['food_id'] + "#" + option_id + "_" + choice_id
        else:
            json_dict['food_id'] = food_dict['food_id'] + "#" + option_id
    return json_util.dumps(json_dict)


def c_food_dict(food_id):
    f_dict = {}
    f_dict['food_id']=food_id
    f_dict['quantity']=int(np.random.choice([1,1,1,1,2,2,3]))
    f_dict['instructions'] = ['Cook with love' if np.random.randint(2) else 'no'][0]
    return f_dict
def nprand(a,b=-1):
    if(b==-1):
        return np.random.randint(a)
    else:
        return np.random.randint(a,b)

In [ ]:
food_embed(c_food_dict(FoodItem.objects[0].id))

In [41]:
order_placement(order)

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [119]:
def generate_order():
    input_order = {'table': random_table(), 'orders': []}
    for n in range(np.random.randint(1, 5)):
        input_order['orders'].append({})
        input_order['orders'][n]['placed_by'] = random_user()
        input_order['orders'][n]['food_list'] = [food_embed(c_food_dict(v)) for v in random_food_list()]
    return input_order

def order_placement(input_order):
    ordered_table = Table.objects.get(id=input_order['table'])
    table_order = TableOrder(table=str(ordered_table.name), table_id=str(ordered_table.name),
                             timestamp=datetime.datetime.now())
    for order in input_order['orders']:
        food_list = [FoodItemMod.from_json(food_dict) for food_dict in order['food_list']]
        table_order.orders.append(
            Order(placed_by=User.objects.get(id=order['placed_by']).to_dbref(), food_list=food_list).save().to_dbref())
        table_order.timestamp = datetime.datetime.now()
        table_order.save()
    ordered_table.update(push__table_orders=table_order.to_dbref())
    Restaurant.objects(tables__in=[str(ordered_table.id)]).update(push__table_orders=table_order.to_dbref())
    return TableOrder.objects.get(id=table_order.id).to_json()


def generate_asstype():
    assist_input = {'table': random_table(), 'user': random_user(),'assistance_type': Assistance.types[np.random.randint(len(Assistance.types))]}
    return assist_input

def assistance_req(assist_input):
    curr_table = Table.objects.get(id=assist_input['table'])
    curr_ass = Assistance(user=User.objects.get(id=assist_input['user']).to_dbref())
    curr_ass.assistance_type = assist_input['assistance_type']
    curr_ass.timestamp = datetime.datetime.now()
    curr_ass.save()
    curr_table.update(push__assistance_reqs=curr_ass.to_dbref())
    return curr_ass

def send_assistance_req(assist_id):
    table = Table.objects(assistance_reqs__in=[assist_id])[0]
    for staff in table.staff:
        if np.random.randint(3):
            time.sleep(1)
            Assistance.objects.get(id=assist_id).update(set__accepted_by=staff.to_dbref())
            return staff.name
    time.sleep(3)
    for staff in Staff.objects:
        if np.random.randint(3):
            time.sleep(1)
            Assistance.objects.get(id=assist_id).update(set__accepted_by=staff.to_dbref())
            return staff.name

In [138]:
order_placement(generate_order())

'{"_id": {"$oid": "5e8e1593d3b24ec580fd83a2"}, "table": "table6", "table_id": "table6", "orders": [{"_id": {"$oid": "5e8e1592d3b24ec580fd83a1"}, "placed_by": {"$oid": "5e8e10bfd3b24ec580fd82f7"}, "food_list": [{"food_id": "5e8e10bed3b24ec580fd828a", "name": "MUTTON GHEE ROAST", "description": "Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee", "price": "370", "instructions": "Cook with love", "quantity": 1, "status": "queued"}, {"food_id": "5e8e10ced3b24ec580fd831a", "name": "HONEY CHILLY POTATO", "description": "Crispy Fried Potatoes Tossed With Chilli And Honey", "price": "190", "instructions": "no", "quantity": 1, "status": "queued"}, {"food_id": "5e8e10bed3b24ec580fd82ad#chicken", "name": "FIERY PENNE PASTA (VEG/ CHICKEN / SHRIMPS)", "description": "NA", "price": "300/340/380", "instructions": "no", "quantity": 2, "status": "queued", "food_options": {"options": {"CHICKEN": "340"}, "choices": []}}, {"food_id": "5e8e10bed3b24ec580fd82c5", "name": "DAL MAKHANI", "des

In [141]:

def non_completed(class_objects, cooking, skip=0):
    if cooking:
        for obj in class_objects:
            if obj.status == 'queued':
                if skip == 0:
                    return obj
                else:
                    skip = 0
                    continue
        return 'all_cooking'
    else:
        for obj in class_objects:
            if obj.status == 'cooking':
                if skip == 0:
                    return obj
                else:
                    skip = 0
                    continue
        return 'all_completed'


def food_status_check(class_objects, skip=0):
    for obj in class_objects:
        if obj.status == 'queued':
            if skip == 0:
                return obj
            else:
                skip = 0
                continue
    return 'all_cooking'


def food_status_check_cook(class_objects, skip=0):
    for obj in class_objects:
        if obj.status == 'cooking':
            if skip == 0:
                return obj
            else:
                skip = 0
                continue
    return 'all_completed'


def pick_order():
    if True:
        table_order = non_completed(TableOrder.objects, True)
        order = non_completed(table_order.orders, True)
        if not isinstance(order, Order):
            TableOrder.objects.get(id=table_order.id).update(set__status='cooking')
            print('changing table order status')
            return pick_order()

        food_ob = food_status_check(order.food_list)
        if isinstance(food_ob, FoodItemMod):
            food_id = food_ob.food_id
            return (str(table_order.id), str(order.id), food_id)
        elif food_ob == 'all_cooking':
            Order.objects.get(id=order.id).update(set__status='cooking')
            print('changing order status')
            return pick_order()


def pick_order2():
    if True:
        table_order = non_completed(TableOrder.objects, False)
        order = non_completed(table_order.orders, False)
        if not isinstance(order, Order):
            TableOrder.objects.get(id=table_order.id).update(set__status='completed')
            return pick_order()

        food_ob = food_status_check_cook(order.food_list)
        if isinstance(food_ob, FoodItemMod):
            food_id = food_ob.food_id
            return (str(table_order.id), str(order.id), food_id)
        elif food_ob == 'all_completed':
            Order.objects.get(id=order.id).update(set__status='completed')
            return pick_order()

In [143]:

def validate_for_order(order_id, cooking=True):
    if cooking:
        for food_item in Order.objects.get(id=order_id).food_list:
            if food_item.status == 'cooking':
                continue
            else:
                return False
        return True
    else:
        for food_item in Order.objects.get(id=order_id).food_list:
            if food_item.status == 'completed':
                continue
            else:
                return False
        return True


def validate_for_table_order(order_id, cooking=True):
    if cooking:
        for order in TableOrder.objects.get(id=order_id).orders:
            if order.status == 'cooking':
                continue
            else:
                return False
        return True
    else:
        for order in TableOrder.objects.get(id=order_id).orders:
            if order.status == 'completed':
                continue
            else:
                return False
        return True


def order_status_cooking(status_tuple):
    order = Order.objects.get(id=status_tuple[1])
    food_json_dict = json_util.loads(order.fetch_food_item(status_tuple[2]))
    food_json_dict['status'] = 'cooking'
    Order.objects.get(id=status_tuple[1]).update(pull__food_list__food_id=FoodItemMod(food_id=status_tuple[2]).food_id)
    Order.objects.get(id=status_tuple[1]).update(push__food_list=FoodItemMod.from_json(json_util.dumps(food_json_dict)))
    if validate_for_order(status_tuple[1], True):
        Order.objects.get(id=status_tuple[1]).update(set__status='cooking')
    if validate_for_table_order(status_tuple[0], True):
        TableOrder.objects.get(id=status_tuple[0]).update(set__status='cooking')
    return "Done"


def order_status_completed(status_tuple):
    order = Order.objects.get(id=status_tuple[1])
    food_json_dict = json_util.loads(order.fetch_food_item(status_tuple[2]))
    food_json_dict['status'] = 'completed'
    Order.objects.get(id=status_tuple[1]).update(pull__food_list__food_id=FoodItemMod(food_id=status_tuple[2]).food_id)
    Order.objects.get(id=status_tuple[1]).update(push__food_list=FoodItemMod.from_json(json_util.dumps(food_json_dict)))
    if validate_for_order(status_tuple[1], False):
        Order.objects.get(id=status_tuple[1]).update(set__status='completed')
    if validate_for_table_order(status_tuple[0], False):
        TableOrder.objects.get(id=status_tuple[0]).update(set__status='completed')
    return "Done"


In [17]:
generate_order()

{'table': '5e8d98739c69d718046f4acf',
 'orders': [{'placedby': '5e8d98739c69d718046f4ae1',
   'foodlist': ['{"name": "MARGHERITA", "description": "An All-Time Favourite Of Pomodoro, Water Buffalo Mozzarella And Basil Chiffonade", "price": "290", "food_id": "5e8d98729c69d718046f4a9c", "quantity": 1, "instructions": "no", "status": "queued"}',
    '{"name": "GARLIC NAAN", "description": "NA", "price": "50", "food_id": "5e8d987f9c69d718046f4b7b", "quantity": 1, "instructions": "Cook with love", "status": "queued"}']}]}

In [139]:
pick_order()

TypeError: tuple expected at most 1 arguments, got 3

In [290]:
TableOrder.drop_collection()

Order.drop_collection()

Restaurant.objects[0].update(set__tableorders=[])

1

In [197]:
send_assistance_req(str(assistance_req(generate_asstype()).id))

'Mohini'

In [118]:
order_placement(generate_order())

ValueError: The source SON object needs to be of type 'dict' but a '<class 'str'>' was found

In [144]:
status_tuple=pick_order()

order_status_cooking(status_tuple)

'Done'

In [264]:
status_tuple=pick_order2()

order_status_completed(status_tuple)

'Done'

# Configuring Restaurant

In [57]:
def configuring_restaurant(message):
    if message['type'] == 'add_tables':
        table_objects = []
        table_dict_list = []
        for table_pair in message['tables']:
            new_table = Table(name=table_pair['table_name'], seats=table_pair['seats']).save()
            table_objects.append(new_table.to_dbref())
            table_dict_list.append({**{'table_id': str(new_table.id)}, **table_pair})
        Restaurant.objects(restaurant_id=message['restaurant_id'])[0].update(push_all__tables=table_objects)
        return {"restaurant_id": message['restaurant_id'], "type": "add_tables", "tables": table_dict_list}
    elif message['type'] == 'add_staff':
        staff_objects = []
        staff_dict_list = []
        for staff_pair in message['staff']:
            new_staff = Staff(name=staff_pair['staff_name']).save()
            staff_objects.append(new_staff.to_dbref())
            staff_dict_list.append({**{'staff_id': str(new_staff.id)}, **staff_pair})
        Restaurant.objects(restaurant_id=message['restaurant_id'])[0].update(push_all__staff=staff_objects)
        return {"restaurant_id": message['restaurant_id'], "type": "add_staff", "staff": staff_dict_list}

In [58]:
message={'restaurant_id' : 'BNGHSR0002',"type":"add_staff","staff":[{"staff_name":"Chikka"}]}

In [59]:
configuring_restaurant(message)

{'restaurant_id': 'BNGHSR0002',
 'type': 'add_staff',
 'staff': [{'staff_id': '5e8dfb85d3b24ec580fd79ba', 'staff_name': 'Chikka'}]}

In [408]:
message

{'restaurant_id': 'BNGHSR0002',
 'type': 'add_servers',
 'servers': [{'name': 'Chikka'}]}

In [390]:
configuring_restaurant({
    'restaurant_id' : 'BNGHSR0002',
    'type':'add_tables',
    'tables':[{'table_name':'Table 6','seats':6},{'table_name':'Table 5','seats':6},{'table_name':'Table 3','seats':6}]
})

[{'table_id': '5e8cb9539c69d718046f4a1f', 'table_name': 'Table 6', 'seats': 6},
 {'table_id': '5e8cb9539c69d718046f4a20', 'table_name': 'Table 5', 'seats': 6},
 {'table_id': '5e8cb9539c69d718046f4a21', 'table_name': 'Table 3', 'seats': 6}]

In [147]:
#Returning queue, cooking and completed lists
q_list = []
cook_list=[]
comp_list =[] 
for table_order_ob in TableOrder.objects:
    tabord_dict=json_util.loads(table_order_ob.to_json())
    if(tabord_dict['status']=='completed'):
        comp_dict.append(tabord_dict)
        break
    for order in tabord_dict['orders']:
        for food_item in order['food_list']:
            if(food_item['status']=='queued'):
                update_list = q_list
                index = check_exists(tabord_dict['_id'],update_list)
                if(index>=0):
                    order_index=check_exists(order['_id'],update_list[index]['orders'])
                    if(order_index>=0):
                        update_list[index]['orders'][order_index]['foodlist'].append(food_item)
                    else:
                        o_app_dict={k:order[k] for k in ['_id', 'placed_by', 'status']}
                        o_app_dict['foodlist']=[]
                        o_app_dict['foodlist'].append(food_item)
                        update_list[index]['orders'].append(o_app_dict)
                        
                else:
                    t_app_dict={k:tabord_dict[k] for k in ['_id', 'table', 'status', 'timestamp']}
                    o_app_dict={k:order[k] for k in ['_id', 'placed_by', 'status']}
                    o_app_dict['foodlist']=[]
                    o_app_dict['foodlist'].append(food_item)
                    t_app_dict['orders']=[]
                    t_app_dict['orders'].append(o_app_dict)
                    update_list.append(t_app_dict)
            elif(food_item['status']=='cooking'):
                update_list = cook_list
                index = check_exists(tabord_dict['_id'],update_list)
                if(index>=0):
                    order_index=check_exists(order['_id'],update_list[index]['orders'])
                    if(order_index>=0):
                        update_list[index]['orders'][order_index]['foodlist'].append(food_item)
                    else:
                        o_app_dict={k:order[k] for k in ['_id', 'placedby', 'status']}
                        o_app_dict['foodlist']=[]
                        o_app_dict['foodlist'].append(food_item)
                        update_list[index]['orders'].append(o_app_dict)
                        
                else:
                    t_app_dict={k:tabord_dict[k] for k in ['_id', 'table', 'status', 'timestamp']}
                    o_app_dict={k:order[k] for k in ['_id', 'placedby', 'status']}
                    o_app_dict['foodlist']=[]
                    o_app_dict['foodlist'].append(food_item)
                    t_app_dict['orders']=[]
                    t_app_dict['orders'].append(o_app_dict)
                    update_list.append(t_app_dict)
            elif(food_item['status']=='completed'):
                update_list = comp_list
                index = check_exists(tabord_dict['_id'],update_list)
                if(index>=0):
                    order_index=check_exists(order['_id'],update_list[index]['orders'])
                    if(order_index>=0):
                        update_list[index]['orders'][order_index]['foodlist'].append(food_item)
                    else:
                        o_app_dict={k:order[k] for k in ['_id', 'placedby', 'status']}
                        o_app_dict['foodlist']=[]
                        o_app_dict['foodlist'].append(food_item)
                        update_list[index]['orders'].append(o_app_dict)
                        
                else:
                    t_app_dict={k:tabord_dict[k] for k in ['_id', 'table', 'status', 'timestamp']}
                    o_app_dict={k:order[k] for k in ['_id', 'placedby', 'status']}
                    o_app_dict['foodlist']=[]
                    o_app_dict['foodlist'].append(food_item)
                    t_app_dict['orders']=[]
                    t_app_dict['orders'].append(o_app_dict)
                    update_list.append(t_app_dict)

KeyError: 'placedby'

# Returning restaurant object

In [404]:
pprint(json_util.loads(Restaurant.objects.exclude('menu','servers')[0].to_json()))

{'_id': ObjectId('5e8d98729c69d718046f4acd'),
 'address': '',
 'assistance_reqs': [],
 'menu': [],
 'name': 'House of Commons',
 'restaurant_id': 'BNGHSR0001',
 'servers': [],
 'tableorders': [],
 'tables': [{'_id': ObjectId('5e8d98729c69d718046f4ace'),
             'assistance_reqs': [],
             'name': 'table1',
             'nofusers': 0,
             'seats': 6,
             'servers': [ObjectId('5e8d98759c69d718046f4ae3'),
                         ObjectId('5e8d98759c69d718046f4ae4')],
             'tableorders': [],
             'users': []},
            {'_id': ObjectId('5e8d98739c69d718046f4acf'),
             'assistance_reqs': [],
             'name': 'table2',
             'nofusers': 0,
             'seats': 6,
             'servers': [ObjectId('5e8d98759c69d718046f4ae3'),
                         ObjectId('5e8d98759c69d718046f4ae4')],
             'tableorders': [],
             'users': []},
            {'_id': ObjectId('5e8d98739c69d718046f4ad0'),
             'assi

In [217]:
Restaurant.objects.exclude('menu')

[<Restaurant: Restaurant object>]

In [45]:
pprint(json_util.loads(TableOrder.objects[0].to_json()))

{'_id': ObjectId('5e89cfb20806f415e7b830b9'),
 'orders': [{'_id': ObjectId('5e89cfb10806f415e7b830b8'),
             'foodlist': [{'description': 'Cottage Cheese Cooked In Spinach '
                                          'Puree, Tempered Cumin And Ginger '
                                          'Garlic Paste',
                           'food_id': '5e89c5079c69d718046f4600',
                           'instructions': 'Cook with love',
                           'name': 'PALAK PANEER',
                           'price': '260',
                           'quantity': 1,
                           'status': 'completed'},
                          {'description': 'Double Chicken, Double Cheese And '
                                          'Topped With A Sunny Side Up Fried '
                                          'Egg, Served With Fries',
                           'food_id': '5e89c5079c69d718046f45dd',
                           'instructions': 'no',
                           

In [2169]:
if(0):
    order=Order.objects.get(id='5e8097106a2fad6b020c61d4')

    order.update(pull__foodlist__food_id=FoodItemMod(food_id='5e7f298b6a2fad6b020c6033').food_id)

    Order.objects.get(id='5e8097106a2fad6b020c61d4').update(push__foodlist=FoodItemMod.from_json(json_util.dumps(food)))

    food=json_util.loads(Order.objects.get(id='5e8097106a2fad6b020c61d4').fetch_fooditem('5e7f298b6a2fad6b020c6033'))

    food['status']='completed'


    food

In [2186]:
json_util.loads(Order.objects.get(id=status_tuple[1]).to_json())

{'_id': ObjectId('5e8097116a2fad6b020c61d8'),
 'placedby': ObjectId('5e7f298d6a2fad6b020c6053'),
 'foodlist': [{'food_id': '5e7f298b6a2fad6b020c5ff7',
   'name': 'CHEESE DYNABITES',
   'description': 'Classic Cheese Balls Served with New Orleans Dip',
   'price': '220',
   'instructions': 'no',
   'quantity': 2,
   'status': 'cooking'},
  {'food_id': '5e7f298b6a2fad6b020c5ffb',
   'name': 'HOC SALAD (VEG)',
   'description': 'Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing',
   'price': '180',
   'instructions': 'Cook with love',
   'quantity': 2,
   'status': 'completed'},
  {'food_id': '5e7f298b6a2fad6b020c6020',
   'name': 'JIANG CHILLI CHICKEN',
   'description': 'Chicken Tossed With Shredded Ginger And Chilli In Jiang Style',
   'price': '260',
   'instructions': 'no',
   'quantity': 1,
   'status': 'completed'}],
 'status': 'pending'}

# Consolidating

## Pages to DF

In [21]:
half_the_menu=[]

for i in range(6):
    half_the_menu.append(pages[i])

half_menu_text="\n\n".join(half_the_menu)

In [22]:
str_menu = pd.DataFrame(columns=['sub_category','name','description','price'])

In [23]:
description=dish_name=price=subcat=''
counter=0
for line in half_menu_text.split("\n\n"):
    if(re.search("^[\$]",line)):
        subcat = line[1:].strip()
    else:
        if(counter%3==0):
            dish_name = line.strip()
        elif(counter%3==1):
            description = line.strip()
        elif(counter%3==2):
            price = line.strip()
            str_menu.loc[str_menu.shape[0]]=[subcat,dish_name,description,price]
        counter+=1

## creating main categories

In [24]:
main_category={"BAR BITES":"BAR BITES","SALADS":"SALADS","VEG STARTERS":"VEG STARTERS","CHICKEN STARTERS":"NON-VEG STARTERS","LAMB STARTERS":"NON-VEG STARTERS","EGG STARTERS":"NON-VEG STARTERS","FROM THE GRILL":"NON-VEG STARTERS","FROM THE CLAY OVEN":"CHANGE"}

In [25]:
str_menu.loc[:,"main_category"]=str_menu['sub_category'].apply(lambda x: main_category[x])

In [26]:
str_menu.loc[27,"main_category"]="VEG STARTERS"

str_menu.loc[28,"main_category"]="VEG STARTERS"

str_menu.loc[29,"main_category"]="VEG STARTERS"

str_menu.loc[30,"main_category"]="VEG STARTERS"

In [27]:
str_menu.loc[str_menu['main_category']=='CHANGE',"main_category"]="NON-VEG STARTERS"

In [28]:
str_menu.columns

Index(['sub_category', 'name', 'description', 'price', 'main_category'], dtype='object')

In [30]:
str_menu=str_menu[['main_category','sub_category', 'name', 'description', 'price']]

## Taking care of multiple prices

In [44]:
lengths = str_menu['price'].str.split('/').map(len)

str_menu = pd.DataFrame({'main_category': np.repeat(str_menu['main_category'], lengths),
                    'sub_category': np.repeat(str_menu['sub_category'], lengths),
                    'name': np.repeat(str_menu['name'],lengths),
                    'description': np.repeat(str_menu['description'],lengths),
                    'price': chainer(str_menu['price'])})

str_menu.reset_index(inplace=True,drop=True)

str_menu.loc[45,'price']="250"

pickle.dump(str_menu,open('half_menu_main.pkl','wb'))

str_menu=pickle.load(open('half_menu_main.pkl','rb'))

## COnverting into json

In [33]:
final_list_json=[]

In [35]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        
        

In [4]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

In [9]:
from pprint import pprint

In [1]:
json.load(Restaurant.objects[0].to_json())

NameError: name 'Restaurant' is not defined

# js.data

In [17]:
pickle.dump(final_json,open('jason.pkl','wb'))

In [11]:
pickle.dump(final_list_json,open('jason.pkl','wb'))